In [1]:
!pip install pyspark
# -------------------------------------------------
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from datetime import datetime, date, timedelta
from dateutil import relativedelta
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.functions import to_timestamp, to_date
from pyspark.sql import functions as F
from pyspark.sql.functions import collect_list, collect_set, concat, first, array_distinct, col, size, expr
from pyspark.sql import DataFrame
import random
import pandas as pd
# --------------------------------------------------------------
spark = SparkSession\
        .builder\
        .getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=0e9c0c72d0d59be1ae54fb2eb8ba7ec25c400f1c4ee778aa42a163e57caf5368
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [4]:
sqlContext = SQLContext(spark)

data_df = sqlContext.read.parquet("/content/drive/MyDrive/Big Data/VenmoSample.snappy.parquet")

data_df.show()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------+-------+----------------+-------------------+--------------------+-----------+--------------------+
|   user1|  user2|transaction_type|           datetime|         description|is_business|            story_id|
+--------+-------+----------------+-------------------+--------------------+-----------+--------------------+
| 1218774|1528945|         payment|2015-11-27 10:48:19|                Uber|      false|5657c473cd03c9af2...|
| 5109483|4782303|         payment|2015-06-17 11:37:04|              Costco|      false|5580f9702b64f70ab...|
| 4322148|3392963|         payment|2015-06-19 07:05:31|        Sweaty balls|      false|55835ccb1a624b14a...|
|  469894|1333620|          charge|2016-06-03 23:34:13|                  🎥|      false|5751b185cd03c9af2...|
| 2960727|3442373|         payment|2016-05-29 23:23:42|                   ⚡|      false|574b178ecd03c9af2...|
| 3977544|2709470|         payment|2016-09-29 22:12:07|          Chipotlaid|      false|57ed2f4723e064eac...|
| 3766386|4

In [5]:
from pyspark.sql import SparkSession
import pandas as pd
import ast

def convert_to_dict(data):
    try:
        return ast.literal_eval(data.replace("u'", "'").replace("'", '"'))
    except (ValueError, SyntaxError):
        return {}

def process_data(spark, filepath, chunksize=50000):
    dfs = []
    for i, chunk in enumerate(pd.read_csv(filepath, delimiter='\t', header=None, error_bad_lines=False, chunksize=chunksize)):
        # Extract 'id' column before processing
        id_col = chunk[0].astype(int)
        
        # Convert data to dictionary and process
        chunk[1] = chunk[1].apply(convert_to_dict)
        chunk = chunk[chunk[1].apply(len) > 0]

        # Normalize each dictionary in the series into a dataframe
        dfs_chunk = [pd.json_normalize(d) if len(d) > 0 else pd.DataFrame() for d in chunk[1]]
        chunk = pd.concat(dfs_chunk, ignore_index=True)
        
        # Re-add the 'id' column to the chunk
        chunk['id'] = id_col

        # Convert pandas dataframe to spark dataframe
        spark_df = spark.createDataFrame(chunk)

        dfs.append(spark_df)

    return dfs

# Initialize Spark session
spark = SparkSession.builder.getOrCreate()

# Load Venmo data
venmo_df = spark.read.parquet("/content/drive/MyDrive/Big Data/VenmoSample.snappy.parquet")

# List of file paths
paths = ['/content/drive/MyDrive/Big Data/Data/face1results.csv', '/content/drive/MyDrive/Big Data/Data/face2results.csv', '/content/drive/MyDrive/Big Data/Data/face3results.csv']
         #'/content/drive/MyDrive/Big Data/Data/face4results.csv', '/content/drive/MyDrive/Big Data/Data/face5results.csv']

face_dfs = []
for path in paths:
    face_dfs.extend(process_data(spark, path, chunksize=50000))

# Now you have Venmo data in `venmo_df`, and Face data in `face_dfs` list


<ipython-input-5-6e9d3ee24a03>:13: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  for i, chunk in enumerate(pd.read_csv(filepath, delimiter='\t', header=None, error_bad_lines=False, chunksize=chunksize)):
Skipping line 215018: expected 2 fields, saw 3

Skipping line 272042: expected 2 fields, saw 3
Skipping line 295806: expected 2 fields, saw 3

Skipping line 516611: expected 2 fields, saw 3

<ipython-input-5-6e9d3ee24a03>:13: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  for i, chunk in enumerate(pd.read_csv(filepath, delimiter='\t', header=None, error_bad_lines=False, chunksize=chunksize)):
Skipping line 26397: expected 2 fields, saw 3

Skipping line 106563: expected 2 fields, saw 3
Skipping line 108494: expected 2 fields, saw 3
Skipping line 113144: expected 2 fields, saw 3
Skipping line 143961: e

In [6]:
# Combine all face data into a single DataFrame
from functools import reduce
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

face_df = unionAll(*face_dfs)

# Now join Venmo data with Face data
combined_df = venmo_df.join(face_df, venmo_df.user1 == face_df.id)

# Now you have a combined DataFrame `combined_df`


In [7]:
shape = (combined_df.count(), len(combined_df.columns))
print(shape)


(185988, 36)


In [ ]:
num_unique_users1 = combined_df.select("user1").distinct().count()
print(num_unique_users1)


In [ ]:
combined_df.show()

In [ ]:
combined_df.createOrReplaceTempView("df_temp")
demographic_venmo = spark.sql("""
    SELECT user1, `attribute.gender.confidence`, `attribute.gender.value` AS gender,
           `attribute.age.value` AS age,
           `attribute.race.confidence`, `attribute.race.value` as race
    FROM df_temp
""")

demographic_venmo.show()

In [ ]:
# Group by "race" column and count occurrences
race_counts = demographic_venmo.groupBy("race").count()

# Display the category counts
race_counts.show()

In [ ]:
# Group by "race" column and count occurrences
gender_counts = demographic_venmo.groupBy("gender").count()

# Display the category counts
gender_counts.show()

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# Assuming you have already loaded the data into a PySpark DataFrame named 'data'

# Define the columns to be one-hot encoded
categorical_columns = ["race", "gender"]

# Create a list to store the stages of the pipeline
stages = []

# Iterate over the categorical columns and perform string indexing and one-hot encoding
for column in categorical_columns:
    # StringIndexer: Converts categorical values to numerical values
    indexer = StringIndexer(inputCol=column, outputCol=column + "_index")
    
    # OneHotEncoder: Performs one-hot encoding on the indexed values
    encoder = OneHotEncoder(inputCols=[indexer.getOutputCol()], outputCols=[column + "_encoded"])
    
    # Add the stages to the pipeline
    stages += [indexer, encoder]

# Create a pipeline with all the stages
pipeline = Pipeline(stages=stages)

# Fit the pipeline to the data
pipeline_model = pipeline.fit(demographic_venmo)

# Transform the data using the pipeline
transformed_data = pipeline_model.transform(demographic_venmo)

# Show the transformed data
transformed_data.show()


In [ ]:
# Drop the 'gender' and 'race' columns
transformed_data = transformed_data.drop("gender", "race")

# Show the updated DataFrame
transformed_data.show()

In [ ]:
# Assuming you have the DataFrame named 'transformed_data'

# Rename the column
transformed_data = transformed_data.withColumnRenamed("attribute.gender.confidence", "attribute_gender_confidence")

# Show the updated DataFrame
transformed_data.show()


In [ ]:
# Assuming you have the DataFrame named 'transformed_data'

# Rename the column
transformed_data = transformed_data.withColumnRenamed("attribute.race.confidence", "attribute_race_confidence")

# Show the updated DataFrame
transformed_data.show()

In [ ]:
spending_venmo = spark.read.parquet("/content/drive/MyDrive/Big Data/Data/spending_behavior_with_y.parquet")

In [ ]:
spending_venmo.createOrReplaceTempView("spending_venmo")

spending_venmo.show(2)

In [ ]:
target_variable = spark.sql("""
    SELECT user1, y
    FROM spending_venmo
""")

target_variable.show()

In [ ]:
joined_df = demographic_venmo.join(target_variable, demographic_venmo.user1 == target_variable.user1, "left")

In [ ]:

# Assuming you have already loaded the DataFrame as 'df'

# Drop duplicates based on all columns
joined_df = joined_df.dropDuplicates()

# Show the updated DataFrame
joined_df.show()


In [ ]:
joined_df.createOrReplaceTempView("behaviorial_with_y")

joined_df.show(2)

In [ ]:
from pyspark.sql.functions import isnan, isnull

# Check for null or NaN values in the target column
null_count = joined_df.filter(isnull("y") | isnan("y")).count()

null_count

In [ ]:
pandas_df = joined_df.toPandas()


In [ ]:
pandas_df.head()

In [ ]:
pandas_df.to_csv('/content/drive/MyDrive/Big Data/Data/behavioural_with_y_all.csv', index=False)


In [ ]:
# Assuming you have a DataFrame named "matchedData"

# Specify the output path for the Parquet file
output_path = "/content/drive/MyDrive/Big Data/Data/face_all_joined_df.parquet"

from pyspark.sql.functions import col

# Rename the existing column
df = joined_df.withColumnRenamed('user1', 'new_column_name')

# Save the DataFrame as a Parquet file
joined_df.write.parquet(output_path)


In [ ]:
joined_df.show(2)